In [2]:
import json
import pandas as pd
from pandasql import sqldf
# # python -m spacy download en
# import spacy
# # from spacy.lang.en.stop_words import STOP_WORDS
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# import numpy as np
# from keras.utils import to_categorical
# from keras.models import Sequential
# from keras.layers import Dense
# from sklearn.metrics import confusion_matrix
# import seaborn as sns
# import matplotlib.pyplot as plt

In [3]:
data_dir = './data/'

In [4]:
data = pd.read_json(data_dir + 'News_Category_Dataset.json', lines=True)

In [5]:
data.describe()

,authors,category,date,headline,link,short_description
count,124989,124989,124989,124989,124989,124989
unique,19250,31,1498,124560,124964,103905
top,,POLITICS,2017-01-27 00:00:00,Sunday Roundup,https://www.huffingtonpost.comhttp://nymag.com...,
freq,14151,32739,100,90,2,19590
first,NaN,NaN,2014-04-18 00:00:00,NaN,NaN,NaN
last,NaN,NaN,2018-05-26 00:00:00,NaN,NaN,NaN


In [6]:
data.head()

,authors,category,date,headline,link,short_description
0,Melissa Jeltsen,CRIME,2018-05-26,There Were 2 Mass Shootings In Texas Last Week...,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...
1,Andy McDonald,ENTERTAINMENT,2018-05-26,Will Smith Joins Diplo And Nicky Jam For The 2...,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.
2,Ron Dicker,ENTERTAINMENT,2018-05-26,Hugh Grant Marries For The First Time At Age 57,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...
3,Ron Dicker,ENTERTAINMENT,2018-05-26,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...
4,Ron Dicker,ENTERTAINMENT,2018-05-26,Julianna Margulies Uses Donald Trump Poop Bags...,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ..."


In [7]:
data.columns

Index(['authors', 'category', 'date', 'headline', 'link', 'short_description'], dtype='object')

In [8]:
data.shape

(124989, 6)

In [17]:
# clean text with spacy
# tfidf with keras
# nn with keras

In [9]:
# sqldf("""select category, count(*) from data group by 1 order by 2 desc""")

In [12]:
# nlp = spacy.load('en')

# docs = data['text'].tolist()

# def token_filter(token):
#     return not (token.is_punct | token.is_space | token.is_stop | len(token.text) <= 4)

# filtered_tokens = []
# for doc in nlp.pipe(docs):
#     tokens = [token.lemma_ for token in doc if token_filter(token) and token.text not in STOP_WORDS]
#     filtered_tokens.append(' '.join(tokens))

# data['clean_text'] = filtered_tokens

In [11]:
# data = pd.read_csv('clean_data.csv')

In [13]:
# encode class values as integers
# encoder = LabelEncoder()
# encoder.fit(data['category'])
# y = encoder.transform(data['category'])

In [14]:
# X_train, X_test, y_train, y_test = train_test_split(data.clean_text, 
#                                                     y, 
#                                                     test_size = 0.2, 
#                                                     random_state = 1, 
#                                                     stratify = y)

# categories = list(np.unique(y))

In [15]:
# tfidf = TfidfVectorizer(ngram_range=(1,1), use_idf=True, analyzer="word")

In [16]:
# X_train = tfidf.fit_transform(X_train.values.astype('U'))
# X_test = tfidf.transform(X_test.values.astype('U'))

# y_train = to_categorical(y_train, num_classes = len(categories))
# y_test = to_categorical(y_test, num_classes = len(categories))

# model = Sequential()
# model.add(Dense(80, input_dim=X_train.shape[1], activation='relu'))
# model.add(Dense(len(categories), activation='sigmoid'))

# model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [1]:
import boto3

In [2]:
s3_resource = boto3.resource('s3')

In [8]:
s3_resource.create_bucket(Bucket='kaggle-progress', CreateBucketConfiguration={'LocationConstraint': 'us-east-2'})

s3.Bucket(name='kaggle-progress')

In [9]:
bkt = s3_resource.Bucket(name='kaggle-progress')

In [11]:
key = s3_resource.key.Key(bkt, 'README.md')

AttributeError: 's3.ServiceResource' object has no attribute 'key'

In [14]:
s3_resource = boto3.client('s3')

In [15]:
s3_resource.list_buckets()

{'ResponseMetadata': {'RequestId': '33163A2A6DBAA7BE',
  'HostId': 'qFfWuP81vmJkUaZCLfXd7JxtA8j2FbvmeMdxq6BRdRBxrwIBiKW39emNskkjRrvMRTcXds+XMn4=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'qFfWuP81vmJkUaZCLfXd7JxtA8j2FbvmeMdxq6BRdRBxrwIBiKW39emNskkjRrvMRTcXds+XMn4=',
   'x-amz-request-id': '33163A2A6DBAA7BE',
   'date': 'Tue, 27 Nov 2018 20:52:35 GMT',
   'content-type': 'application/xml',
   'transfer-encoding': 'chunked',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'Buckets': [{'Name': 'elasticbeanstalk-us-east-2-343056730941',
   'CreationDate': datetime.datetime(2018, 3, 3, 18, 22, tzinfo=tzutc())},
  {'Name': 'kaggle-progress',
   'CreationDate': datetime.datetime(2018, 11, 27, 20, 46, 24, tzinfo=tzutc())},
  {'Name': 'molurus-api',
   'CreationDate': datetime.datetime(2018, 3, 4, 16, 7, 55, tzinfo=tzutc())}],
 'Owner': {'ID': 'a230f932c29b32dec38f43cf7f8137fec1100cac70b80638abb3aa7f0158a4df'}}

In [16]:
s3 = boto3.client('s3')

In [20]:
s3.upload_file('hello.txt', 'kaggle-progress', 'hello.txt')

In [18]:
import os

In [19]:
os.listdir()

['eda.ipynb', '.DS_Store', '.ipynb_checkpoints', 'data']

In [22]:
s3.download_file('kaggle-progress', 'hello.txt', 'hello.txt')

In [ ]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/guide/s3-example-creating-buckets.html